# Housing Unit Allocation Full Workflow

## Overview
Given a building inventory that uses IN-CORE standard columns, run housing unit allocation.
This process checks to see if the housing unit inventory is available and if not it will create it.
This process checks to see if the address point inventory is available and if not it will create it.

With the housing unit inventory and address point inventory created, they will be uploaded to IN-CORE Dataservice.

With the required files on IN-CORE Dataservice, the housing unit allocation method will run.
Functions are provided to obtain and clean data required for the version 2 Housing Unit Allocation. 

## Required Inputs
Program requires the following inputs:
1. Building inventory file from pyincore
    - IN-CORE account
    
## Output Description
The output of this workflow is a CSV file with the housing unit inventory allocated to a building inventory using the housing unit allocation model.

The output CSV is designed to be used in the Interdependent Networked Community Resilience Modeling Environment (IN-CORE).

IN-CORE is an open source python package that can be used to model the resilience of a community. To download IN-CORE, see:

https://incore.ncsa.illinois.edu/


## Instructions
Users can run the workflow by executing each block of code in the notebook.

## Description of Program
- program:    ncoda_07cv1_run_HUA_workflow
- task:       Start with building inventory and run housing unit allocation algorithm
- See github commits for description of program updates
- Current Version:    2022-08-29 - v2 workflow
- 2022-10-06 - clean up code and test output for Salt Lake City
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE), Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

- Suggested Citation:
Rosenheim, Nathanael (2021) “Detailed Household and Housing Unit Characteristics: Data and Replication Code.” DesignSafe-CI. 
https://doi.org/10.17603/ds2-jwf6-s535.

## Setup Python Environment

In [1]:
import pandas as pd
import geopandas as gpd # For reading in shapefiles
import numpy as np
import sys # For displaying package versions
import os # For managing directories and file paths if drive is mounted

from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingdamage.buildingdamage import BuildingDamage

from pyincore_viz.geoutil import GeoUtil as viz

In [2]:
# https://ipyleaflet.readthedocs.io/en/latest/api_reference/heatmap.html
import ipyleaflet as ipylft
from ipyleaflet import Map, Heatmap, LayersControl

In [3]:
# To reload submodules need to use this magic command to set autoreload on
%load_ext autoreload
%autoreload 2
# open, read, and execute python program with reusable commands
from pyncoda.ncoda_00b_directory_design import directory_design
from pyncoda.ncoda_07a_generate_hui import generate_hui_functions
from pyncoda.ncoda_07c_generate_addpt import generate_addpt_functions
from pyncoda.ncoda_07d_run_hua_workflow import hua_workflow_functions


In [4]:
import scooby # Reports Python environment

In [5]:
# Generate report of Python environment
print(scooby.Report(additional=['pandas','pyincore','pyincore_viz','ipyleaflet']))


--------------------------------------------------------------------------------
  Date: Fri Oct 14 15:44:52 2022 Central Daylight Time

                OS : Windows
            CPU(s) : 12
           Machine : AMD64
      Architecture : 64bit
               RAM : 31.6 GiB
       Environment : Jupyter

  Python 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:50:36) [MSC
  v.1929 64 bit (AMD64)]

            pandas : 1.5.0
          pyincore : 1.7.0
      pyincore_viz : Version unknown
        ipyleaflet : 0.17.1
             numpy : 1.23.3
             scipy : 1.9.1
           IPython : 8.5.0
        matplotlib : 3.6.0
            scooby : 0.5.12
--------------------------------------------------------------------------------


In [6]:
# Check working directory - good practice for relative path access
os.getcwd()

'c:\\Users\\nathanael99\\MyProjects\\IN-CORE\\Tasks\\PublishHUIv2\\HousingUnitInventories_2022-03-03\\ReplicationCode\\intersect-community-data'

In [7]:
# Edit Data Dictionary for Community

# Example of data dictionary for one community with one county
# Check how to capitalize the state name at
## https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/

# NOTE on file path length. WINDOWS has a limit of 260 characters for file path length.
# Community name needs to be short to avoid this limit.

'''
communities = {'Mobile_AL' : {
                    'community_name' : 'Mobile, AL',
                    'STATE' : 'ALABAMA',
                    'years' : ['2010'],
                    'counties' : { 
                        1 : {'FIPS Code' : '01097', 'Name' : 'Mobile County, AL'}}}}
'''


communities = {'SLC_UT' : {
                    'community_name' : 'Salt Lake City, UT',
                    'STATE' : 'UTAH',
                    'years' : ['2010'],
                    'counties' : { 
                        1 : {'FIPS Code' : '49035', 'Name' : 'Salt Lake City County, UT'}}}}

## Step 1: Set up pyincore and read in data
IN-CORE is an open source python package that can be used to model the resilience of a community. To download IN-CORE, see:

https://incore.ncsa.illinois.edu/

Registration is free.

In [8]:
client = IncoreClient()
# IN-CORE caches files on the local machine, it might be necessary to clear the memory
client.clear_cache() 

Connection successful to IN-CORE services. pyIncore version detected: 1.7.0


In [9]:
# create data_service object for loading files
data_service = DataService(client)

## Read in Housing Unit Inventory or create a new one

In [10]:
version = '2.0.0'
version_text = 'v2-0-0'

# Save Outputfolder - due to long folder name paths output saved to folder with shorter name
# files from this program will be saved with the program name - 
# this helps to follow the overall workflow
outputfolder = "OutputData"

# Set random seed for reproducibility
seed = 1000
basevintage = 2010

generate_hui_df = generate_hui_functions(
                    communities =   communities,
                    seed =          seed,
                    version =       version,
                    version_text=   version_text,
                    basevintage=    basevintage,
                    outputfolder=   outputfolder
                    )

hui_dataset_id = generate_hui_df.generate_hui_v2_for_incore()

Generating Housing Unit Inventory v2.0.0 data for Salt Lake City, UT
Connection successful to IN-CORE services. pyIncore version detected: 1.7.0
Number of datasets matching Housing Unit Inventory v2.0.0 data for Salt Lake City, UT: 1
Dataset Housing Unit Inventory v2.0.0 data for Salt Lake City, UT already exists in IN-CORE
Dataset already exists in IN-CORE with filename hui_v2-0-0_SLC_UT_2010_rs1000.csv
Use dataset_id: 63487fd188a94f621d8fd4a0
Dataset already exists on IN-CORE, use dataset_id: 63487fd188a94f621d8fd4a0


In [11]:
# Housing Unit inventory
housing_unit_inv_id = hui_dataset_id
# load housing unit inventory as pandas dataframe
housing_unit_inv = Dataset.from_data_service(housing_unit_inv_id, data_service)
filename = housing_unit_inv.get_file_path('csv')
print("The IN-CORE Dataservice has saved the Housing Unit Inventory on your local machine: "+filename)

The IN-CORE Dataservice has saved the Housing Unit Inventory on your local machine: C:\Users\nathanael99\.incore\cache_data\648c22dd689743dc9dedf242006adc5a9c21de9a89c016323dccfedef94f7737\63487fd188a94f621d8fd4a0\hui_v2-0-0_SLC_UT_2010_rs1000\hui_v2-0-0_SLC_UT_2010_rs1000.csv


In [12]:
housing_unit_inv_df = pd.read_csv(filename, header="infer")

In [13]:
housing_unit_inv_df['huid'].describe()

count                   364210
unique                  364210
top       B490351001001006H001
freq                         1
Name: huid, dtype: object

## Read in Building Inventory

In [14]:
# Building inventory
bldg_inv_id = "62fea288f5438e1f8c515ef8" # SLC building inventory - Milad Roohi
# load building inventory
bldg_inv = Dataset.from_data_service(bldg_inv_id, data_service)
filename = bldg_inv.get_file_path('shp')
print("The IN-CORE Dataservice has saved the Building Inventory on your local machine: "+filename)

The IN-CORE Dataservice has saved the Building Inventory on your local machine: C:\Users\nathanael99\.incore\cache_data\648c22dd689743dc9dedf242006adc5a9c21de9a89c016323dccfedef94f7737\62fea288f5438e1f8c515ef8\Salt Lake County All Building\Salt Lake County All Building.shp


In [15]:
bldg_inv_gdf = gpd.read_file(filename)
# Check CRS of building inventory
bldg_inv_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
from pyproj import CRS
# Update CRS to EPSG:4326 if not already in that format
if bldg_inv_gdf.crs != CRS.from_epsg(4326):
    bldg_inv_gdf = bldg_inv_gdf.to_crs(epsg=4326)
    print("The CRS of the building inventory has been updated to EPSG:4326")
else:
    print("The CRS of the building inventory is already in EPSG:4326")

The CRS of the building inventory is already in EPSG:4326


In [17]:
# Check Unique ID
bldg_inv_gdf[['guid','bldg_id']].astype(str).describe().T

,count,unique,top,freq
guid,284925,284925,6f8c3468-4696-4371-bc91-3a7ceec1e768,1
bldg_id,284925,112,16200000000000.0,17249


In [18]:
bldg_inv_gdf.head(1).T

,0
parid,28200000000000.0
parid_card,28200000000000.0
bldg_id,28200000000000.0
struct_typ,RM1L
archetype,COM3
str_prob,None
year_built,1993
no_stories,1
a_stories,None
b_stories,None


In [19]:
# What variable has information to determine if building is residential or not?
# County observations by variable
bldg_inv_gdf[['archetype','occ_type','guid']].groupby(['archetype','occ_type']).count()

,,guid
archetype,occ_type,
AGR1,AGR1,57
COM1,COM1,7900
COM10,COM10,373
COM2,COM2,4674
COM3,COM3,2096
COM4,COM4,5674
COM5,COM5,270
COM6,COM6,217
COM7,COM7,421


#### HAZUS Occupancy Status
Table 3-1: Hazus Occupancy Class Definitions

| Label             | Hazus Occupancy Class            |
|-------------------|----------------------------------|
|                   | Residential                      |
| RES1              | Single Family Dwelling           |
| RES2              | Mobile / Manufactured Home       |
| RES3A            | RES3A Duplex                      |
| RES3B            | RES3B 3-4 Units                   |
| RES3C  |  5-9 Units                 |
| RES3D  |  10-19 Units                         |
| RES3E  |   20-49 Units                      |
| RES3F |   50+ Units                       |
| RES4              | Temporary Lodging                |
| RES5              | Institutional Dormitory          |
| RES6              | Nursing Home                     |
|-------|------------------------------------|
|                   | Commercial                       |
| COM1              | Retail Trade                     |
| COM2              | Wholesale Trade                  |
| COM3              | Personal and Repair Services     |
| COM4              | Professional/Technical Services  |
| COM5  | Banks                              |
| COM6  | Hospital                           |
| COM7  | Medical Office/Clinic              |
| COM8  | Entertainment & Recreation         |
| COM9  | Theaters                           |
| COM10 | Parking                            |
|------|------------------------|
|       | Industrial                         |
|------|------------------------|
| IND1  | Heavy                              |
| IND2  | Light                              |
| IND3  | Food/Drugs/Chemicals               |
| IND4  | Metals/Minerals Processing         |
| IND5  | High Technology                    |
| IND6  | Construction                       |
|------|------------------------|
|       | Agriculture                        |
|------|------------------------|
| AGR1  | Agriculture                        |
|       | Religion/Membership Organizations  |
| REL1  | Church/Nonprofit                   |
|------|------------------------|
|       | Government                         |
|------|------------------------|
| GOV1  | General Services                   |
| GOV2  | Emergency Response                 |
|------|------------------------|
|      | Education              |
|------|------------------------|
| EDU1 | Grade School, Library  |
| EDU2 | College/University     |


### Find Spatial Extent of Building Inventory
Need to know which counties are included in the building inventory.

In [20]:
#map = viz.plot_gdf_map(bldg_inv_gdf,column='dlevel')
#map

#### Use Tract ID to find county FIPS code
The first 5 digits of the tract ID are the FIPS code for the county.

In [21]:
bldg_inv_gdf['tract_id'].describe()

count    2.849250e+05
mean     4.903511e+10
std      4.125693e+03
min      4.903510e+10
25%      4.903511e+10
50%      4.903511e+10
75%      4.903511e+10
max      4.903511e+10
Name: tract_id, dtype: float64

In [22]:
# Substring of tractid to get county 5 digit fips code 
# covert tract_id to string with leading zeros
bldg_inv_gdf['tract_id_str'] = bldg_inv_gdf['tract_id'].astype(str).str.zfill(11)
bldg_inv_gdf['county'] = bldg_inv_gdf['tract_id_str'].str[:5]
bldg_inv_gdf[['tract_id_str','county']].describe()

,tract_id_str,county
count,284925,284925
unique,193,1
top,49035113104,49035
freq,8331,284925


### Building Inventory File has 1 county
The Salt Late City building inventory only has 1 county.

## Generate Address Point Inventory


In [23]:
Nofal_residential_archetypes = { 
        1 : 'One-story sf residential building on a crawlspace foundation',
        2 : 'One-story mf residential building on a slab-on-grade foundation',
        3 : 'Two-story sf residential building on a crawlspace foundation',
        4 : 'Two-story mf residential building on a slab-on-grade foundation'}

# HAZUS Archetypes for residential buildings inlcude an estimate of housing units
HAZUS_residential_archetypes = { 
    "RES1" : {'Description' : "Single Family Dwelling", 'HU estimate' : 1},
    "RES2" : {'Description' : "Mobile / Manufactured Home", 'HU estimate' : 1},
    "RES3A" : {'Description' : "Duplex", 'HU estimate' : 2},
    "RES3B" : {'Description' : "3-4 Units", 'HU estimate' : 3},
    "RES3C" : {'Description' : "5-9 Units", 'HU estimate' : 7},
    "RES3D" : {'Description' : "10-19 Units", 'HU estimate' : 15},
    "RES3E" : {'Description' : "20-49 Units", 'HU estimate' : 30},
    "RES3F" : {'Description' : "50+ Units", 'HU estimate' : 50},
    "RES4" : {'Description' : "Temporary Lodging", 'HU estimate' : 1},
    "RES5" : {'Description' : "Institutional Dormitory", 'HU estimate' : 1},
    "RES6" : {'Description' : "Nursing Home", 'HU estimate' : 1}
    }

residential_archetypes = HAZUS_residential_archetypes 

In [24]:
for community in communities.keys():
    print("Address point inventory for: "+community)
    print("Based on building inventory: "+bldg_inv_id)
    generate_addpt_df = generate_addpt_functions(
                        community =   community,
                        communities = communities,
                        hui_df = housing_unit_inv_df,
                        bldg_inv_gdf = bldg_inv_gdf,
                        bldg_inv_id = bldg_inv_id,
                        residential_archetypes = residential_archetypes,
                        archetype_var = 'archetype',
                        building_area_var = 'sq_foot',
                        building_area_cutoff = 300,
                        seed =          seed,
                        version =       version,
                        version_text=   version_text,
                        basevintage=    basevintage,
                        outputfolder=   outputfolder
                        )

    addpt_dataset_id = generate_addpt_df.generate_addpt_v2_for_incore()

Address point inventory for: SLC_UT
Based on building inventory: 62fea288f5438e1f8c515ef8
***************
Address Point Inventory Workflow
***************

Generating Address Point Inventory v2.0.0 data for SLC_UT 2010
Connection successful to IN-CORE services. pyIncore version detected: 1.7.0
Number of datasets matching Address Point Inventory v2.0.0 data for SLC_UT 2010: 1
Dataset Address Point Inventory v2.0.0 data for SLC_UT 2010 already exists in IN-CORE
Dataset already exists in IN-CORE with filename addpt_v2-0-0_SLC_UT_2010_62fea288f5438e1f8c515ef8.csv
Use dataset_id: 63497ce4b2332d481c436c09
Dataset already exists on IN-CORE, use dataset_id: 63497ce4b2332d481c436c09


### Read in Address Point Inventory
The address point inventory is an intermediate file based on the building inventory. The address point inventory acts as the bridge between the building inventory and the housing unit inventory.

In [25]:
# Check if addpt_dataset_id is string
if isinstance(addpt_dataset_id, str):
    print("The Address Point Inventory ID is a pandas string")
    # Address Point inventory
    addpt_inv_id = addpt_dataset_id
    # load housing unit inventory as pandas dataframe
    addpt_inv = Dataset.from_data_service(addpt_inv_id, data_service)
    filename = addpt_inv.get_file_path('csv')
    print("The IN-CORE Dataservice has saved the Address Point Inventory on your local machine: "+filename)
    addpt_inv_df = pd.read_csv(filename, header="infer")
# else if addpt_dataset_id is a dataframe
elif isinstance(addpt_dataset_id, pd.DataFrame):
    addpt_inv_df = addpt_dataset_id
    print("The Address Point Inventory ID contains a pandas dataframe")
else:
    print("The Address Point Inventory is not a string or pandas dataframe")

The Address Point Inventory ID is a pandas string
The IN-CORE Dataservice has saved the Address Point Inventory on your local machine: C:\Users\nathanael99\.incore\cache_data\648c22dd689743dc9dedf242006adc5a9c21de9a89c016323dccfedef94f7737\63497ce4b2332d481c436c09\addpt_v2-0-0_SLC_UT_2010_62fea288f5438e1f8c515ef8\addpt_v2-0-0_SLC_UT_2010_62fea288f5438e1f8c515ef8.csv


In [26]:
#addpt_inv_df = pd.read_csv(filename, header="infer")
addpt_inv_df['addrptid'].describe()

count                                             502643
unique                                            502643
top       STc64d6569-ff3e-410d-a1ad-213af6ba463cAP000000
freq                                                   1
Name: addrptid, dtype: object

## Step 2: Housing Unit Allocation v2

### Setup Housing Unit Allocation

### Run Housing Unit Allocation

In [27]:
for community in communities.keys():
    print("Housing Unit Allocation for: "+community)
    print("Based on housing unit inventory: "+hui_dataset_id)
    print("Based on building inventory: "+bldg_inv_id)

    outputfolders = directory_design(state_county_name = community,
                                            outputfolder = outputfolder)

    run_hua_gdf = hua_workflow_functions(
                        community =   community,
                        hui_df = housing_unit_inv_df,
                        bldg_gdf = bldg_inv_gdf,
                        addpt_df = addpt_inv_df,
                        seed =          seed,
                        version =       version,
                        version_text=   version_text,
                        basevintage=    basevintage,
                        outputfolder=   outputfolder,
                        outputfolders = outputfolders
                        )

    hua_gdf = run_hua_gdf.housing_unit_allocation_workflow()

Housing Unit Allocation for: SLC_UT
Based on housing unit inventory: 63487fd188a94f621d8fd4a0
Based on building inventory: 62fea288f5438e1f8c515ef8
Running up Housing Unit Allocation for SLC_UT
The file already exists: c:\Users\nathanael99\MyProjects\IN-CORE\Tasks\PublishHUIv2\HousingUnitInventories_2022-03-03\ReplicationCode\intersect-community-data/OutputData/SLC_UT/hua_v2-0-0_SLC_UT_2010_rs1000.csv


c:\Users\nathanael99\MyProjects\IN-CORE\Tasks\PublishHUIv2\HousingUnitInventories_2022-03-03\ReplicationCode\intersect-community-data\pyncoda\ncoda_07d_run_hua_workflow.py:426: DtypeWarning: Columns (0,17,24,27,31) have mixed types. Specify dtype option on import or set low_memory=False.
  huav2_df = pd.read_csv(csv_filepath)


Converting geometry to Geodataframe


## Explore and Validate Housing Unit Allocation


In [28]:
hua_gdf.head()

,huid,Block2010,blockid,bgid,tractid,FIPScounty,numprec,ownershp,race,hispan,...,strctid_Block2010_flagsetrm,addrptid,guid,huestimate,huicounter_addpt,placeNAME10,x,y,archetype,geometry
0,B490351001001006H001,490351001001006,4.903510e+14,4.903510e+11,4.903510e+10,49035.0,1.0,2.0,1.0,0.0,...,1.0,ST30ef8530-dfda-46ef-96c8-17e8fced0537AP000000,30ef8530-dfda-46ef-96c8-17e8fced0537,1.0,1.0,Salt Lake City,-111.917033,40.804688,RES5,POINT (-111.91703 40.80469)
1,B490351001001006H002,490351001001006,4.903510e+14,4.903510e+11,4.903510e+10,49035.0,1.0,2.0,1.0,0.0,...,1.0,STe176ee84-c869-47fe-a825-9854d9cb2bcaAP000001,e176ee84-c869-47fe-a825-9854d9cb2bca,15.0,2.0,Salt Lake City,-111.916801,40.804502,RES3D,POINT (-111.91680 40.80450)
2,B490351001001006H003,490351001001006,4.903510e+14,4.903510e+11,4.903510e+10,49035.0,1.0,2.0,1.0,0.0,...,1.0,STe176ee84-c869-47fe-a825-9854d9cb2bcaAP000002,e176ee84-c869-47fe-a825-9854d9cb2bca,15.0,3.0,Salt Lake City,-111.916801,40.804502,RES3D,POINT (-111.91680 40.80450)
3,B490351001001006H005,490351001001006,4.903510e+14,4.903510e+11,4.903510e+10,49035.0,1.0,2.0,1.0,0.0,...,2.0,STe176ee84-c869-47fe-a825-9854d9cb2bcaAP000003,e176ee84-c869-47fe-a825-9854d9cb2bca,15.0,4.0,Salt Lake City,-111.916801,40.804502,RES3D,POINT (-111.91680 40.80450)
4,B490351001001006H006,490351001001006,4.903510e+14,4.903510e+11,4.903510e+10,49035.0,1.0,2.0,1.0,0.0,...,2.0,STe176ee84-c869-47fe-a825-9854d9cb2bcaAP000005,e176ee84-c869-47fe-a825-9854d9cb2bca,15.0,6.0,Salt Lake City,-111.916801,40.804502,RES3D,POINT (-111.91680 40.80450)


In [29]:
hua_gdf.head(1).T

,0
huid,B490351001001006H001
Block2010,490351001001006
blockid,490351001001006.0
bgid,490351001001.0
tractid,49035100100.0
FIPScounty,49035.0
numprec,1.0
ownershp,2.0
race,1.0
hispan,0.0


In [30]:
# describe the housing unit allocation primary keys and foreign keys
hua_gdf[['huid','Block2010','addrptid','guid','strctid']].astype(str).describe().T

,count,unique,top,freq
huid,394422,364211,nan,30212
Block2010,394422,11631,999999999999999,30212
addrptid,394422,364211,nan,30212
guid,394422,284926,nan,20524
strctid,394422,255375,nan,30212


## Merge Housing Unit Allocation with Housing Unit Inventory

In [31]:
housing_unit_inv_df.head()

,huid,blockid,bgid,tractid,FIPScounty,numprec,ownershp,race,hispan,family,vacancy,gqtype,incomegroup,hhinc,randincome,poverty
0,B490351001001006H001,490351001001006,490351001001,49035100100,49035,1,2.0,1.0,0.0,0.0,0,0,12,4,84651.0,0.0
1,B490351001001006H002,490351001001006,490351001001,49035100100,49035,1,2.0,1.0,0.0,0.0,0,0,8,3,44695.0,0.0
2,B490351001001006H003,490351001001006,490351001001,49035100100,49035,1,2.0,1.0,0.0,0.0,0,0,10,3,56215.0,0.0
3,B490351001001006H004,490351001001006,490351001001,49035100100,49035,1,2.0,1.0,0.0,0.0,0,0,14,5,128776.0,0.0
4,B490351001001006H005,490351001001006,490351001001,49035100100,49035,1,2.0,1.0,0.0,0.0,0,0,7,3,37398.0,0.0


In [32]:
housing_unit_inv_df[['huid','blockid']].astype(str).describe()

,huid,blockid
count,364210,364210
unique,364210,11630
top,B490351001001006H001,490351011021001
freq,1,667


In [33]:
# Merge HUA with HUI
hua_cols = ['huid','guid','placeNAME10','x','y']
hua_hui_df = pd.merge(left = housing_unit_inv_df,
                       right = hua_gdf[hua_cols],
                       on='huid',
                       how='left')

In [34]:
hua_hui_df[['huid','blockid','guid']].astype(str).describe()

,huid,blockid,guid
count,364210,364210,364210
unique,364210,11630,254714
top,B490351001001006H001,490351011021001,nan
freq,1,667,20524


In [35]:
# Replace missing guid 
hua_hui_df['guid'] = hua_hui_df['guid'].fillna('missing building id')

In [36]:
hua_hui_df[['huid','blockid','guid']].astype(str).describe()

,huid,blockid,guid
count,364210,364210,364210
unique,364210,11630,254714
top,B490351001001006H001,490351011021001,missing building id
freq,1,667,20524


## Save Housing Unit Allocation to CSV and Upload to IN-CORE Dataservice

In [37]:
# save hua_hui_gdf to csv
check_folder = outputfolder
output_filename = f'hua_{version_text}_{community}_{basevintage}_rs{seed}_{bldg_inv_id}'
csv_filepath = check_folder+"/"+output_filename+'.csv'
savefile = sys.path[0]+"/"+csv_filepath
# Resave results for community name
hua_hui_df.to_csv(savefile, index=False)

In [38]:
# make a county list for community
county_list = ''
for county in communities[community]['counties'].keys():
    state_county = communities[community]['counties'][county]['FIPS Code']
    state_county_name  = communities[community]['counties'][county]['Name']
    print(state_county_name,': county FIPS Code',state_county)
    county_list = county_list + state_county_name+': county FIPS Code '+state_county
county_list

Salt Lake City County, UT : county FIPS Code 49035


'Salt Lake City County, UT: county FIPS Code 49035'

In [39]:
title = "Housing Unit Allocation v2.0.0 data for "+community + " " + str(basevintage)
title

'Housing Unit Allocation v2.0.0 data for SLC_UT 2010'

In [40]:
run_hua_gdf.upload_hua_file_to_incore(title =title,
                        county_list = county_list,
                        csv_filepath = csv_filepath,
                        output_filename = output_filename)

Connection successful to IN-CORE services. pyIncore version detected: 1.7.0
dataset is created with id 6349caa8b2332d481c436c39
The file(s): hua_v2-0-0_SLC_UT_2010_rs1000_62fea288f5438e1f8c515ef8 have been uploaded to IN-CORE
Dataset now on IN-CORE, use dataset_id: 6349caa8b2332d481c436c39
Dataset is only in personal account, contact IN-CORE to make public


'6349caa8b2332d481c436c39'

### Look at population characteristics and compare to US Census

In [ ]:
# convert hui_hua_df to geodataframe
crs = "EPSG:4326"
hua_hui_gdf = gpd.GeoDataFrame(hua_hui_df, 
    geometry=gpd.points_from_xy(hua_hui_df.x, hua_hui_df.y), crs=crs)

In [ ]:
from pyncoda.ncoda_04c_poptableresults import *

In [ ]:
PopResultsTable.pop_results_table(
                  input_df = housing_unit_inv_df, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = "Salt Lake County UT",
                  when = "2010",
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

In [ ]:
# add category for missing building id
buildingdata_conditions = {'cat_var' : {'variable_label' : 'Building Data Availability',
                         'notes' : 'Does Housing Unit have building data?'},
              'condition_list' : {
                1 : {'condition': "(df['guid'] == 'missing building id')", 'value_label': "0 Missing Building Data"},
                2 : {'condition': "(df['guid'] != 'missing building id')", 'value_label': "1 Building Data Available"}}
            }
hua_hui_df = PopResultsTable.add_label_cat_conditions_df(hua_hui_df, conditions = buildingdata_conditions)

In [ ]:

PopResultsTable.pop_results_table(hua_hui_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = "Salt Lake County UT",
                  when = "2010",
                  row_index = "Race Ethnicity",
                  col_index = 'Building Data Availability',
                  row_percent = '0 Missing Building Data')

In [ ]:
PopResultsTable.pop_results_table(hua_hui_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = "Salt Lake County UT",
                  when = "2010",
                  row_index = "Group Quarters Type",
                  col_index = 'Building Data Availability',
                  row_percent = '0 Missing Building Data')

In [ ]:
PopResultsTable.pop_results_table(hua_hui_gdf, 
                   who = "Median Household Income", 
                  what = "by Race, Ethnicity",
                  where = "Salt Lake County, UT",
                  when = "2010",
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

#### Validate the Housing Unit Allocation has worked
Notice that the population count totals for the community should match (pretty closely) data collected for the 2010 Decennial Census. This can be confirmed by going to data.census.gov

Total Population by Race and Ethnicity: https://data.census.gov/cedsci/table?g=0500000US49035&tid=DECENNIALSF12010.P5

Median Income by Race and Ethnicity:

All Households: https://data.census.gov/cedsci/table?g=0500000US49035&tid=ACSDT5Y2012.B19013
Black Households: https://data.census.gov/cedsci/table?g=0500000US49035&tid=ACSDT5Y2012.B19013B
White, not Hispanic Households: https://data.census.gov/cedsci/table?g=0500000US49035&tid=ACSDT5Y2012.B19013H
Hispanic Households: https://data.census.gov/cedsci/table?g=0500000US49035&tid=ACSDT5Y2012.B19013I
Differences in the housing unit allocation and the Census count may be due to differences between political boundaries and the building inventory. See Rosenheim et al 2019 for more details.

The housing unit allocation, plus the building results will become the input for the social science models such as the population dislocatoin model.

### Explore 1 block

In [ ]:
from pyncoda.ncoda_04b_foliummaps import *

In [ ]:
gdfvar_count_df = count_gdfvar_by_building(hua_hui_gdf, blocknum='490351003061000', gdvar='huid')
gdfvar_count_df.head()

In [ ]:
map_selected_block(hua_hui_gdf, blocknum='490351003061000')


In [ ]:
# explore building inventory
addpt_inv_df.head(1).T

In [ ]:
map_selected_block(addpt_inv_df, blocknum='490351003061000',
                        gdfvar = "addrptid",
                        laryername = "Address Points per building")


### Explore Prison Population

In [ ]:
# Check missing building data for correctional facilities
condition1 = (hua_hui_gdf['gqtype'] == 1)
gdf1 = hua_hui_gdf.loc[condition1].copy()
gdf1

In [ ]:
folium_marker_layer_map(gdf = gdf1,
                        gdfvar="Building Data Availability",
                        layername = "Building Data Availability for Prisons",
                        color_levels = ["0 Missing Building Data", "1 Building Data Available"])

In [ ]:
# make list of unique guid for correctional facilities
guid_list = gdf1['guid'].unique().tolist()
# drop missing building id from list
guid_list.remove('missing building id')
guid_list

In [ ]:
# keep guid list in address point inventory
condition2 = (addpt_inv_df['guid'].isin(guid_list))
gdf2 = addpt_inv_df.loc[condition2].copy()
gdf2

In [ ]:
# make a list of unique archetypes 
archetype_list = gdf2['archetype'].unique().tolist()

In [ ]:
folium_marker_layer_map(gdf = gdf2,
                        gdfvar="archetype",
                        layername = "Address Points for Prisons",
                        color_levels = archetype_list)

In [ ]:
# Block 490351116003000 has 2,079 prison population
map_selected_block(addpt_inv_df, blocknum='490351116003000',
                        gdfvar = "addrptid",
                        laryername = "Address Points per building")

In [ ]:
# Block 490351116003000 has 2,079 prison population
map_selected_block(hua_hui_gdf, blocknum='490351116003000')

It looks like the building inventory does not include the main prisons in the county.

## Explore the HUA Results

In [ ]:
# What location should the map be centered on?
center_x = hua_hui_gdf.bounds.minx.mean()
center_y = hua_hui_gdf.bounds.miny.mean()
center_x, center_y



In [ ]:
# visualize population by income and tenure status
lowincome_renters_gdf = hua_hui_gdf.loc[(hua_hui_gdf.hhinc < 3) &
                                    (hua_hui_gdf['ownershp'] == 2)].copy(deep=True)
map = viz.plot_gdf_map(lowincome_renters_gdf,column='hhinc', category=True)
map

In [ ]:
map = viz.plot_gdf_map(hua_hui_gdf,column='hhinc', category=True)
map

In [ ]:
popdata = lowincome_renters_gdf[['y','x','numprec']].values.tolist()
map = Map(center=[center_y,center_x], zoom=11)

low_income_renters = Heatmap(
    locations = popdata,
    radius = 10, 
    max_val = 1000, 
    blur = 10, 
    gradient={0.2: 'yellow', 0.5: 'orange', 1.0: 'red'},
    name = 'Low Income Renters',
)

map.add_layer(low_income_renters);
control = LayersControl(position='topright')
map.add_control(control)
map

In [ ]:
numprec_gdf = hua_hui_gdf.loc[(hua_hui_gdf.numprec >= 0)].copy(deep=True)
popdata = numprec_gdf[['y','x','numprec']].values.tolist()
map = Map(center=[center_y,center_x], zoom=11)

popmap = Heatmap(
    locations = popdata,
    radius = 10, 
    max_val = 10000, 
    blur = 10, 
    gradient={0.2: 'yellow', 0.5: 'orange', 1.0: 'red'},
    name = 'Population Heat Map',
)

map.add_layer(popmap)
control = LayersControl(position='topright')
map.add_control(control)
map


In [ ]:
race1 = hua_hui_gdf.loc[(hua_hui_gdf.race == 1) & 
                    (hua_hui_gdf.hispan == 0) &
                    (hua_hui_gdf.hhinc == 5) &
                    (hua_hui_gdf.ownershp == 1) ].copy(deep=True)
# print how many people are in the race1 dataframe
print("map 1 has ",race1['numprec'].sum()," people")

popdata1 = race1[['y','x','numprec']].values.tolist()
race2 = hua_hui_gdf.loc[(hua_hui_gdf.race == 2) & 
                    (hua_hui_gdf.hispan == 0) &
                    (hua_hui_gdf.hhinc == 1) &
                    (hua_hui_gdf.ownershp == 2) ].copy(deep=True)
# print how many people are in the race1 dataframe
print("map 2 has ",race2['numprec'].sum()," people")
popdata2 = race2[['y','x','numprec']].values.tolist()

map = Map(center=[center_y,center_x], zoom=11)

popmap1 = Heatmap(
    locations = popdata1,
    radius = 10, 
    max_val = 10000, 
    blur = 10, 
    gradient={0.2: 'yellow', 0.5: 'orange', 1.0: 'red'},
    name = 'White, High Income, Owner'
)

popmap2 = Heatmap(
    locations = popdata2,
    radius = 10, 
    max_val = 10000, 
    blur = 10, 
    gradient={0.2: 'green', 0.5: 'blue', 1.0: 'purple'},
    name = 'Black Low Income, Renter'
)

map.add_layer(popmap1)
map.add_layer(popmap2)
control = LayersControl(position='topright')
map.add_control(control)
map


In [ ]:
race1 = hua_hui_gdf.loc[(hua_hui_gdf.race == 1) & 
                    (hua_hui_gdf.hispan == 0) ].copy(deep=True)
# print how many people are in the race1 dataframe
print("map 1 has ",race1['numprec'].sum()," people")

popdata1 = race1[['y','x','numprec']].values.tolist()
race2 = hua_hui_gdf.loc[(hua_hui_gdf.race == 2) & 
                    (hua_hui_gdf.hispan == 0) ].copy(deep=True)
# print how many people are in the race1 dataframe
print("map 2 has ",race2['numprec'].sum()," people")
popdata2 = race2[['y','x','numprec']].values.tolist()

map = Map(center=[center_y,center_x], zoom=11)

popmap1 = Heatmap(
    locations = popdata1,
    radius = 10, 
    max_val = 10000, 
    blur = 10, 
    gradient={0.2: 'yellow', 0.5: 'orange', 1.0: 'red'},
    name = 'White, Not Hispanic'
)

popmap2 = Heatmap(
    locations = popdata2,
    radius = 10, 
    max_val = 10000, 
    blur = 10, 
    gradient={0.2: 'green', 0.5: 'blue', 1.0: 'purple'},
    name = 'Black, Not Hispanic'
)

map.add_layer(popmap1)
map.add_layer(popmap2)
control = LayersControl(position='topright')
map.add_control(control)
map


In [ ]:
#racedot_map(hua_gdf,column='race', category=True)

In [ ]:
popdata = hua_hui_gdf.loc[(hua_hui_gdf.race >= 1)].copy(deep=True)
df_wm = popdata.to_crs(epsg=3857)
ax = df_wm.plot(figsize=(10, 10), column='race',
                      categorical=True, legend=True, markersize = 0.001)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels)

In [ ]:
popdata = hua_hui_gdf.loc[(hua_hui_gdf.race >= 1) &
                       (hua_hui_gdf.placeNAME10 == "Salt Lake City") ].copy(deep=True)
df_wm = popdata.to_crs(epsg=3857)
ax = df_wm.plot(figsize=(10, 10), column='race',
                      categorical=True, legend=True, markersize = 0.01)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels)